# Import libraries

In [1]:
# drive access
from google.colab import drive
drive.mount('/content/drive')

# standard library
import numpy as np
import pandas as pd

# for audio
import librosa
from IPython.display import Audio

# for file streaming
import os

# for model
!pip install transformers
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
import torch

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 59.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 122.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.0 MB/s eta 0:00:00


In [2]:
# load model and tokenizer
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Load examples from bdl

In [5]:
audio, _ = librosa.load('/content/drive/MyDrive/266/Data/Raw_Data/Arctic/cmu_us_bdl_arctic/wav/arctic_b0539.wav', sr=16000)

In [6]:
Audio(audio, rate=16000)

In [7]:
audio, _ = librosa.load('/content/drive/MyDrive/266/Data/Raw_Data/Arctic/cmu_us_bdl_arctic/wav/arctic_b0538.wav', sr=16000)

In [8]:
Audio(audio, rate=16000)

# get the files from bea_Amused folder & create auto transcription using wav2vec

In [9]:
audio_filename = os.listdir('/content/drive/MyDrive/266/Data/Raw_Data/Arctic/cmu_us_bdl_arctic/wav')

audio_filename

['arctic_a0004.wav',
 'arctic_a0050.wav',
 'arctic_a0102.wav',
 'arctic_a0072.wav',
 'arctic_a0124.wav',
 'arctic_a0145.wav',
 'arctic_a0131.wav',
 'arctic_a0108.wav',
 'arctic_a0106.wav',
 'arctic_a0148.wav',
 'arctic_a0019.wav',
 'arctic_a0063.wav',
 'arctic_a0109.wav',
 'arctic_a0059.wav',
 'arctic_a0137.wav',
 'arctic_a0092.wav',
 'arctic_a0158.wav',
 'arctic_a0144.wav',
 'arctic_a0122.wav',
 'arctic_a0083.wav',
 'arctic_a0007.wav',
 'arctic_a0129.wav',
 'arctic_a0116.wav',
 'arctic_a0061.wav',
 'arctic_a0097.wav',
 'arctic_a0024.wav',
 'arctic_a0272.wav',
 'arctic_a0281.wav',
 'arctic_a0269.wav',
 'arctic_a0312.wav',
 'arctic_a0177.wav',
 'arctic_a0234.wav',
 'arctic_a0245.wav',
 'arctic_a0259.wav',
 'arctic_a0280.wav',
 'arctic_a0211.wav',
 'arctic_a0165.wav',
 'arctic_a0207.wav',
 'arctic_a0173.wav',
 'arctic_a0323.wav',
 'arctic_a0224.wav',
 'arctic_a0291.wav',
 'arctic_a0161.wav',
 'arctic_a0251.wav',
 'arctic_a0194.wav',
 'arctic_a0293.wav',
 'arctic_a0317.wav',
 'arctic_a025

In [10]:
audio_df = pd.DataFrame()

audio_df['filename'] = audio_filename

audio_df.head()

,filename
0,arctic_a0004.wav
1,arctic_a0050.wav
2,arctic_a0102.wav
3,arctic_a0072.wav
4,arctic_a0124.wav


In [11]:
auto_transcription = []

for filename in audio_df['filename']:
  audio, _ = librosa.load(f'/content/drive/MyDrive/266/Data/Raw_Data/Arctic/cmu_us_bdl_arctic/wav/{filename}', sr=16000)
  input_value = processor(audio, return_tensors="pt", padding="longest", sampling_rate=16000).input_values
  predicted_id = torch.argmax(model(input_value).logits, dim=-1)
  transcription = processor.batch_decode(predicted_id)
  auto_transcription.append(transcription[0])
  del audio, input_value, predicted_id, transcription

auto_transcription

["LORD BUT I'M GLAD TO SEE YOU AGAIN PHIL",
 'IN SPITE OF THEIR ABSURDITY THE WORDS AFFECTED PHILIP CURIOUSLY',
 'HE WILL FOLLOW US SOON',
 "BUT WHO WAS ILEN'S DOUBLE",
 'IT WAS JEAN SINGING SOFTLY OVER BEYOND THE ROCKS',
 'I HAVE TO BE CAREFUL OF THEM AS THEY TEAR VERY EASILY',
 'PROVIDENCE HAD DELIVERED HIM THROUGH THE MAILSTRUM',
 'HE WADED INTO THE EDGE OF THE WATER AND BEGAN SCRUBBING HIMSELF',
 'THE EMOTION WHICH SHE HAD SUPPRESSED BURST FORTH NOW IN A CHOKING SOB',
 'NOW THESE THINGS HAD BEEN STRUCK DEAD WITHIN HIM',
 'I FOLLOWED THE LINE OF THE PROPOSED RAILROAD LOOKING FOR CHANCES',
 'YES IT WAS A MAN WHO ASKED A STRANGER',
 'DO YOU KNOW THAT YOU ARE SHAKING MY CONFIDENCE IN YOU',
 'HIS IMMACULATE APPEARANCE WAS GONE',
 'THEN HE STEPPED BACK WITH A LOW CRY OF PLEASURE',
 'HE WENT DOWN IN MIDSTREAM SEARCHING THE SHADOWS OF BOTH SHORES',
 'DOES THAT LOOK GOOD',
 'BUT THERE WAS SOMETHING EVEN MORE STARTLING THAN THIS RESEMBLANCE',
 'TWO YEARS AGO I GAVE UP CIVILIZATION FOR THIS',

confirm all files from bea_amused have been auto transcribed & add to the df

In [12]:
assert len(audio_df) == len(auto_transcription)

In [13]:
audio_df['auto_transcription'] = auto_transcription

In [14]:
audio_df.head()

,filename,auto_transcription
0,arctic_a0004.wav,LORD BUT I'M GLAD TO SEE YOU AGAIN PHIL
1,arctic_a0050.wav,IN SPITE OF THEIR ABSURDITY THE WORDS AFFECTED...
2,arctic_a0102.wav,HE WILL FOLLOW US SOON
3,arctic_a0072.wav,BUT WHO WAS ILEN'S DOUBLE
4,arctic_a0124.wav,IT WAS JEAN SINGING SOFTLY OVER BEYOND THE ROCKS


take a look at one example

In [15]:
example = audio_df['filename'].iloc[0]
audio, _ = librosa.load(f'/content/drive/MyDrive/266/Data/Raw_Data/Arctic/cmu_us_bdl_arctic/wav/{example}', sr=16000)

In [16]:
Audio(audio, rate=16000)

In [17]:
audio_df['auto_transcription'].iloc[0]

"LORD BUT I'M GLAD TO SEE YOU AGAIN PHIL"

save the df to csv file for future use

In [18]:
audio_df.to_csv('/content/drive/MyDrive/266/Data/Clean_Data/Arctic/auto_transcription/bdl.csv', index=False)

In [19]:
check = pd.read_csv('/content/drive/MyDrive/266/Data/Clean_Data/Arctic/auto_transcription/bdl.csv')

check.head()

,filename,auto_transcription
0,arctic_a0004.wav,LORD BUT I'M GLAD TO SEE YOU AGAIN PHIL
1,arctic_a0050.wav,IN SPITE OF THEIR ABSURDITY THE WORDS AFFECTED...
2,arctic_a0102.wav,HE WILL FOLLOW US SOON
3,arctic_a0072.wav,BUT WHO WAS ILEN'S DOUBLE
4,arctic_a0124.wav,IT WAS JEAN SINGING SOFTLY OVER BEYOND THE ROCKS
